<a href="https://colab.research.google.com/github/ganamir/ParPy/blob/main/ParPy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import math
import numpy as np
from itertools import combinations
import pandas as pd

df = pd.read_csv("/content/WeightedUnifrac_PCA_Structure.csv")
pca = pd.read_csv("/content/WeightedUnifrac_Explained_Variance.csv")

def calculate_pairwise_combinations(row):
    columns = [col for col in row.index if col.startswith('PC')]
    pairs = list(combinations(columns, 2))
    combinations_values = [(row[pair[0]], row[pair[1]]) for pair in pairs]
    return combinations_values

pairwise_combination = df.apply(calculate_pairwise_combinations, axis=1)

pairwise_df = pd.DataFrame(pairwise_combination, index = df.index)
pairwise_df['Clone'] = df['Clone']
pairwise_df['Treatment'] = df['Treatment']

# Group by 'Clone' column
grouped_df = pairwise_df.groupby('Clone').apply(lambda x: x.drop(['Clone', 'Treatment'], axis=1).stack()).reset_index(level=1, drop=True).reset_index()

# Rename the columns
grouped_df.columns = ['Clone', 'Pair', 'Value']



dot_rule = []
mag1 = []
mag2 = []
dg = []
for i in range(0, len(grouped_df['Value']), 2):
  if i + 1 < len(grouped_df['Value']):
    m_values = []
    for n in range (0, len(grouped_df['Value'][0])):
      dot = np.dot(grouped_df['Value'][i][n], grouped_df['Value'][i+1][n])
      mg1 = np.linalg.norm(grouped_df['Value'][i][n])
      mg2 = np.linalg.norm(grouped_df['Value'][i+1][n])
      mg_total = mg1 * mg2
      cosine = (dot / mg_total)
      angle = math.acos(cosine)
      degree = math.degrees(angle)
      m_values.append(angle)
    # dot_rule.append(dot)
    # mag1.append(mg1)
    # mag2.append(mg2)
    dg.append(m_values)

df = pd.DataFrame()
df['angle-rad'] = dg

pairwise_pca = pd.DataFrame()
pca_pairwise_combination = pca.apply(calculate_pairwise_combinations, axis = 1)
pairwise_pca['Weights'] = pca_pairwise_combination

def multiply_tuples(tuples_list):
    multiplied_tuples = [(x * y) for x, y in tuples_list]
    return multiplied_tuples

pairwise_pca['Multiplied'] = pairwise_pca['Weights'].apply(multiply_tuples)


sum_sines_list = []
sum_cos_list = []
for i in range(0, len(df['angle-rad'])):
  sum_sines = []
  sum_cos = []
  for n in range(0, len(df['angle-rad'][i])):
    weighted_sum_sines = pairwise_pca['Multiplied'][0][n] * math.sin(df['angle-rad'][i][n])
    weighted_cos_sines = pairwise_pca['Multiplied'][0][n] * math.cos(df['angle-rad'][i][n])
    sum_sines.append(weighted_sum_sines)
    sum_cos.append(weighted_cos_sines)
  sum_sines_list.append(sum_sines)
  sum_cos_list.append(sum_cos)

df_rad_sum = pd.DataFrame({'sin': sum_sines_list})
df_rad_sum['cos'] = sum_cos_list

def calculate_sum(lst):
  return sum(lst)

summed_df = df_rad_sum.applymap(calculate_sum)
summed_df['weighted_avg_angle'] = summed_df.apply(lambda row: math.atan2(row['sin'], row['cos']), axis=1)
summed_df['weighted_avg_angle_deg'] = summed_df['weighted_avg_angle'].apply(math.degrees)
summed_df['weighted_avg_angle_deg'] %= 360
summed_df['Clone'] = grouped_df.groupby('Clone')['Clone'].first().reset_index(drop=True)

summed_df.to_csv("DegreeOfParallelism.csv", index = False)